In [3]:
from dataloader import get_loader
import numpy as np
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis



In [2]:
# Load in data
# root = '/Volumes/Datasets/inner_speech/derivatives/'
root =  'dataset/derivatives/' # -sil
creater = get_loader(root)
xn, yn = creater.load_multiple_subjects([1, 2, 3, 4, 5, 6, 7])
xt, yt = creater.load_single_subject(8)

# LDA

In [6]:
# Vectorize data so that X is of size = (n_samples, n_features)
train_x = xn.reshape(xn.shape[0], xn.shape[1]*xn.shape[2])
test_x = xt.reshape(xt.shape[0], xt.shape[1]*xt.shape[2])
print("train_x: ", train_x.shape)
print("test_x: ", test_x.shape)


# Get Labels
train_y = yn[:,1]
test_y = yt[:,1]
print("train_y: ", train_y.shape)
print("test_y: ", test_y.shape)

train_x:  (1080, 147584)
test_x:  (160, 147584)
train_y:  (1080,)
test_y:  (160,)


In [ ]:
# Fit LDA
# solver='svd', shrinkage=None, priors=None, n_components=None, store_covariance=False, tol=0.0001, covariance_estimator=None
clf = LinearDiscriminantAnalysis(solver='svd')
clf.fit(train_x, train_y)



In [12]:
# Predict on test data
predicitons = clf.predict(test_x) 
accuracy = clf.score(test_x, test_y) 
print("Accuracy: ", accuracy*100)

Accuracy:  29.375


In [14]:
# Tranform data to LDA space
transformed_xt = clf.transform(test_x)
transformed_xt.shape

(160, 3)